In [ ]:
import mediapipe as mp
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
from mpl_toolkits.mplot3d import Axes3D


## For Google Collab

from google.colab import drive
drive.mount('/content/drive')

##### Clear Temporary Folder which we will use further

In [ ]:
def clearMesh():
        folder_path = "mesh_frames"

        files = glob.glob(os.path.join(folder_path, '*'))

        for file in files:
                os.remove(file)


In [ ]:
#mediapipe object
face_mesh = mp.solutions.face_mesh.FaceMesh()

#video file
input_video_path = #enter the path of your video input

#input video object
cap = cv2.VideoCapture(input_video_path)

#input video properties
width = int(cap.get(3))
height = int(cap.get(4))
fps = int(cap.get(5))


#temporary folder to store frames
output_folder = 'mesh_frames'
os.makedirs(output_folder, exist_ok=True)

#processing each frame
while True:
    ret, frame = cap.read()
    if not ret:
        print('Video processing complete.')
        break

    #mediapipe requires RGB format
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    #landmarks acquired in results
    results = face_mesh.process(frame_rgb)

    #if no face detected
    if not results.multi_face_landmarks:
        continue

    #extracting landmarks
    landmarks_3d = []
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for landmark in face_landmarks.landmark:
                x, y, z = landmark.x, landmark.y, landmark.z
                landmarks_3d.append((x, y, z))

    landmarks_3d = np.array(landmarks_3d)

    #Delaunay triangulation
    triangulation = Delaunay(landmarks_3d[:, :2])

    #naming the frame to save into temporary folder
    frame_filename = f"{output_folder}/frame_{int(cap.get(cv2.CAP_PROP_POS_FRAMES))}.png"

    # 3D surface plot
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(1, 2, 1, projection='3d')

    # triangulation in 3D
    ax.plot_trisurf(landmarks_3d[:, 0], landmarks_3d[:, 1], landmarks_3d[:, 2],
                    triangles=triangulation.simplices, color='lightblue',edgecolor='none', antialiased=False, shade=150) 

    ax.set_title('3D triangulated mesh')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')


    ax.view_init(elev=-90, azim=-90)

    # image with facial keypoints
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.imshow(frame_rgb)
    ax2.set_title('Face input')
    ax2.axis('off')
    plt.axis('off')
    ax.set_axis_off()
    fig.savefig(frame_filename)
    plt.close()


cap.release()
cv2.destroyAllWindows()





input_folder = 'mesh_frames'


image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg'))]
image_files.sort(key=lambda x: int(x.split('_')[1].split('.')[0]))


first_image = cv2.imread(os.path.join(input_folder, image_files[0]))
height, width, _ = first_image.shape


output_video_path = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = 24
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))


for image_file in image_files:
    image_path = os.path.join(input_folder, image_file)
    frame = cv2.imread(image_path)
    out.write(frame)


out.release()


cap = cv2.VideoCapture(output_video_path)


while True:
    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow('Video', frame)
    if cv2.waitKey(30) & 0xFF == 27:
        break


cap.release()
cv2.destroyAllWindows()

print(f"Video playback complete.")
